In [1]:
from openai import OpenAI
import time
client = OpenAI()
# print(client.api_key)

In [2]:
#上传文件
file = client.files.create(
  file=open("3716.docx", "rb"),
  purpose='assistants'
)

print(file)

FileObject(id='file-LS40LWmKHATw9NCaOIvJIfNI', bytes=739585, created_at=1705390322, filename='3716.docx', object='file', purpose='assistants', status='processed', status_details=None)


In [3]:
instructions = f"""
You are a resume analysis assistant that can parse resumes based on the files I uploaded. 
"""

In [4]:
#创建assistant
assistant = client.beta.assistants.create(
  instructions=instructions,
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

# my_assistant = client.beta.assistants.retrieve(assistant.id)
print(assistant)

Assistant(id='asst_c8BJXTFncsHWXo0FSPXw5y8Q', created_at=1705390330, description=None, file_ids=['file-LS40LWmKHATw9NCaOIvJIfNI'], instructions='\nYou are a resume analysis assistant that can parse resumes based on the files I uploaded. \n', metadata={}, model='gpt-3.5-turbo-1106', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')])


In [5]:
prompt = f"""
Your task is to parse the uploaded file, which is a resume. \
You have to extract the content and output it in json format. \
json should contains several Fields: name, school, phone, educational, experience.\
The educational field represents educational experience and academic qualifications information,\
If no relevant content is found, this field can be empty.\
The experience field represents work experience,\
If no relevant content is found, this field can be empty.
"""

In [6]:
# 创建一个thread（交互）
thread = client.beta.threads.create(
    messages=[
    {
      "role": "user",
      "content":prompt,
      "file_ids": [file.id]
    }
  ]
)

print(thread)

Thread(id='thread_Alk5IIGcgzo1p84gcI7PeBOZ', created_at=1705390346, metadata={}, object='thread')


In [7]:
#和大模型对话
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [8]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status not in ["queued", "in_progress"]:
        break
    time.sleep(1)

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
text = messages.data[0].content[0].text.value
print(text)